# Mapping galaxies' 3D positions

In this notebook, we'll use a bunch of data downloaded from SDSS -- the Sloan Digital Sky Server. We'll organize it, convert redshift to distance, and then export it the standard file type used by planetarium software so that we can look at the galaxies' 3D positions.

First, let's again import all of the modules we will need.

In [7]:
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance, Angle
from astropy.table import Table, Column
from astropy.io import ascii
import numpy as np
import seaborn as sns
from astropy.cosmology import Planck15
sns.set()
%matplotlib inline

### "Querying" SDSS
Next, we have to query our data from SDSS using CasJobs. "Querying" is a term for running a script that will download, from a larger data set, only the data you need. On the database we can see all the different parameters that are available to be queried. We will download the ones necessary to locate its position (RA, Dec, and redshift -- which will give us distance), and some properties of the galaxies, like spectro_Flux_r / i (flux in red / infrared band). The following SQL script will select these data for us.
```
SELECT gz.ra, gz.dec, sp.z, sp.spectroFlux_r as r, 
  sp. spectroFlux_i as i into mydb.galaxydata from DR10.zooSpec as gz 
  JOIN  DR8.specObjAll as sp on sp.specobjid=gz.specobjid
WHERE sp.zWarning=0 and sp.z >0.0005
```
You don't have to do anything with this^. Just download the table: galaxies_archiekinnane.fit.

### Reading in the fits file
We want to open, inspect, and read in the data from the fits file, just like we did with the data in the Hubble Diagram notebook. See if you can remember how to open and name a fits file, look at its "info", and print its columns! Feel free to consult your Hubble Diagram notebook.

In [2]:
# open and name the file, look at info, print columns



### Putting it in an astropy Table
Now put it in a neat astropy Table and take a look at what we have.

In [3]:
# create and name a Table with the fits data

# close the hdu file

# look at your table

### Converting redshift to distance

Remember the linear relationship between redshift and distance that we discovered by creating our Hubble Diagram? Now, we want to convert our redshifts into comoving distances. 

#### What is comoving distance?
There are a few ways to represent distances in Astronomy. 

*Lookback-time distance* is how long it took the light we are seeing to reach us. 
*Luminosity distance* is the distance -- found using the inverse square law -- that the galaxies would emit at their observed brightness (this is what we used in our Hubble Diagram exercise). 
*Comoving distance*, what we're using here, scales out the expansion of the universe. If we were able to stop the expansion of the universe and go and measure the distance to galaxies this is what we would get.

There's a a handy Astropy method called Planck15.comoving_distance that will do this for us. It works by taking redshift as its argument and then returning a distance! Remember how you added columns in the Hubble Diagram exercise? Here, we want to add a column -- name it 'distance' -- where we convert every redshift into a distance. See if you can figure out how to do this below.

In [6]:
# add 'distance' column



### Converting to magnitudes
We also want to convert our luminosity units (nanomaggies) into typical magnitudes. The formula to do so is:
```
Magnitude = 22.5 - 2.5 * log(L in nanomaggies)
```
You might notice that this means that the brighter something is (higher nanomaggies) the lower magnitude it has. This is an extremely annoying astronomy convention.

Add a column below based on this conversion. Name it 'mags'.

### Using SkyCoord
We also want to create a column of coordinate objects, which will make it much easier to convert between coordinate systems later. To do this, we're going to use the astropy method SkyCoord. You can input 'locations' to SkyCoord using whatever coordinate system you currently have your data in -- for us, that means RA, Dec, and comoving distance. Then, you just have to specify what it is that you gave SkyCoord, which we do using "frame = 'icrs'", which is basically the J2000 equatorial frame -- what our RA and Dec values are referential to. Run the code below:
```
coordsCol = SkyCoord(your_tablename['ra'], your_tablename['dec'],unit=(u.degree, u.degree),distance=Distance(your_tablename['distance'],u.pc),frame='icrs')

```

In [5]:
# create coordsCol with SkyCoord



To see an example of what SkyCoord can do, run the following code, which performs the normally very difficult task of converting coordinate systems like *that*!

```
print (coordsCol[1])
print (coordsCol[1].galactic)
```

### Plotting our galaxies on the sky
Let's see where our galaxies are on the sky. It's kind of complicated to change to a sky projection instead of just a normal rectangular plot, so I'm just going to have you run the code below. Look through it and see if you can figure out what's going on, though! We're utilizing our SkyCoord objects again to make unit conversions really easily.
```
fig = plt.figure (figsize=(13,6))
ax = fig.add_subplot(111,projection="mollweide")
plt.scatter(Angle(galaxies['ra'],u.deg).wrap_at(180.*u.deg).radian, Angle(galaxies['dec'],u.deg).radian,
            s=0.1*np.log(galaxies['r']))
```

Finally, we're going to write our data to a VOtable,a standard format that is read by most planetarium software. The code below will create this file in whatever directory you're currently in, and then we can use this to look at our galaxies' 3D positions using some fancy software!
```
your_tablename.write('galaxydata.xml',format='votable')
```

In [9]:
# write your VOtable



That's it!